In [1]:
import pandas as pd
import os
import sys

In [2]:
QUALITY_POSITIVE_MIN_SCORE = 80
QUALITY_NEGATIVE_MIN_SCORE = 50
SENTIMENT_POSITIVE_MIN_SCORE = 70
SENTIMENT_NEGATIVE_MIN_SCORE = 30
SPEAKER_1_NAME = "SPEAKER 1"
SPEAKER_2_NAME = "SPEAKER 2"

cwd = os.path.join(os.getcwd(), "working_dir")
sentiments_path = os.path.join(cwd, "sentiments")
scorecard_path = os.path.join(cwd, "scorecards")
OVERVIEW_DIR = os.path.join(cwd, "overview")

if not os.path.exists(OVERVIEW_DIR):
    os.mkdir(OVERVIEW_DIR)

In [4]:
def sentiment_score(sentiment_df):
    """
    pass like this sentiment_df[sentiment_df["Speaker"]=="SPEAKER 1"]
    """
    pos_neg_count = sentiment_df.groupby("sentiment").Speaker.count()
    return round((pos_neg_count.POSITIVE/(pos_neg_count.POSITIVE+pos_neg_count.NEGATIVE))*100, 2)

def get_overall_sentiment(score, positive_thres=70, negative_thres=30):
    return "POSITIVE" if score > positive_thres else ("NEGATIVE" if score < negative_thres else "NEUTRAL")

def get_overview(score_df, sentiment_df):
    global filename
    score_df.loc["Total Score", "sub_parameter"] = "Total Score"
    score_df = score_df[["sub_parameter", "score"]].set_index("sub_parameter").transpose()
    score_df["quality_feedback"] = get_overall_sentiment(score_df["Total Score"][0], QUALITY_POSITIVE_MIN_SCORE, QUALITY_NEGATIVE_MIN_SCORE)
    score_df.insert(0, "filename",  filename)
    score_df["call_duration"] = sentiment_df.End.iloc[-1]
    score_df[f"{SPEAKER_1_NAME} Sentiment Score %"] = sentiment_score(sentiment_df[sentiment_df["Speaker"]==SPEAKER_1_NAME])
    score_df[f"{SPEAKER_1_NAME} Sentiment"] = get_overall_sentiment(score_df[f"{SPEAKER_1_NAME} Sentiment Score %"][0], SENTIMENT_POSITIVE_MIN_SCORE, SENTIMENT_NEGATIVE_MIN_SCORE)
    score_df[f"{SPEAKER_2_NAME} Sentiment Score %"] = sentiment_score(sentiment_df[sentiment_df["Speaker"]==SPEAKER_2_NAME])
    score_df[f"{SPEAKER_2_NAME} Sentiment"] = get_overall_sentiment(score_df[f"{SPEAKER_2_NAME} Sentiment Score %"][0], SENTIMENT_POSITIVE_MIN_SCORE, SENTIMENT_NEGATIVE_MIN_SCORE)
    score_df["OVERALL Sentiment Score %"] = sentiment_score(sentiment_df)
    score_df["OVERALL Sentiment"] = get_overall_sentiment(score_df["OVERALL Sentiment Score %"][0],SENTIMENT_POSITIVE_MIN_SCORE, SENTIMENT_NEGATIVE_MIN_SCORE)
    return score_df


df = pd.DataFrame()
sentiment_files = [i.rsplit(os.extsep, 1)[0] for i in os.listdir(sentiments_path)]
dfs = []
if not sentiment_files:
    print(f"No csv files found in sentiments folder..")
    sys.exit()
for scorecard_file in os.listdir(scorecard_path):
    filename, ext = scorecard_file.rsplit(os.extsep, 1)
    print(f"Processing {filename}..")
    if not filename in sentiment_files:
        print(f"{filename} not found in sentiments folder, please run the sentiment_analysis script first..")
        continue
    score_df = pd.read_csv(os.path.join(scorecard_path, filename+".csv"), index_col=1)
    sentiment_df = pd.read_csv(os.path.join(sentiments_path, filename+".csv"), index_col=0)
    try:
        dfs.append(get_overview(score_df, sentiment_df))
    except Exception as e:
        print(e)
        continue
pd.concat(dfs).reset_index(drop=True).to_csv(os.path.join(OVERVIEW_DIR, "scorecard_overview.csv"))
print(f"Saved overview in overview folder..")

Processing InboundSampleRecording..
Processing OUTBOUNDSAMPLE_01..
Processing OUTBOUNDSAMPLE_02..
Saved overview in overview folder..
